In [1]:
!pip install ncps PyWavelets


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install tensorflow==2.10


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, Add, Flatten, Dense
from tensorflow import keras
from tensorflow.keras.initializers import lecun_normal
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import pandas as pd
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from ncps import wirings
from ncps.tf import CfC, LTC
from scipy.signal import stft
from scipy.signal import butter, filtfilt, resample
import pywt
from scipy import signal
import os
import csv
import matplotlib.pyplot as plt

import scipy.io
#from tqdm import tqdm
import argparse
import gc

In [10]:

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print("GPU detected: ", physical_devices)
else:
    print("No GPU detected.")

wiring = wirings.AutoNCP(30, 4)
ncp = LTC(wiring, return_sequences=False)

if activation == 'selu':
    initializer = keras.initializers.lecun_normal(seed=42)
if activation == 'tanh':
    initializer = 'glorot_uniform'
else:
    initializer = 'he_normal'

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(None,  62, 64, 5)))
model.add(keras.layers.ConvLSTM2D(filters=16,kernel_size=(12, 3),padding='same',strides=(1,2),activation=activation,
        dropout=dr, recurrent_dropout=dr, return_sequences=True, kernel_initializer=initializer))

model.add(keras.layers.TimeDistributed(Flatten()))
model.add(Dropout(dr))
model.add(keras.layers.TimeDistributed(Dense(75, activation=activation, kernel_initializer=initializer)))
model.add(ncp)
model.add(keras.layers.Activation("softmax"))


No GPU detected.


NameError: name 'activation' is not defined

In [24]:
import os

session = 1
path = r"C:\Users\ASUS\Desktop\BTP\eeg_raw_data"  # Use raw string or double backslashes

# Construct the full path
session_path = os.path.join(path, str(session))

# Check if the directory exists
if os.path.exists(session_path):
    persons_list = os.listdir(session_path)
    print("Files:", persons_list)
else:
    print("Error: Directory does not exist:", session_path)

y = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]

Files: ['10_20151014.mat', '11_20150916.mat', '12_20150725.mat', '13_20151115.mat', '14_20151205.mat', '15_20150508.mat', '1_20160518.mat', '2_20150915.mat', '3_20150919.mat', '4_20151111.mat', '5_20160406.mat', '6_20150507.mat', '7_20150715.mat', '8_20151103.mat', '9_20151028.mat']


In [25]:
import os
import scipy.io
import numpy as np

# Define the directory where the .mat files are stored
data_dir = r"C:\Users\ASUS\Desktop\BTP\eeg_feature_smooth\1"  # Corrected path for Windows

print(f"Checking directory: {os.path.abspath(data_dir)}")
print(f"Does directory exist? {os.path.exists(data_dir)}")

# Sample persons_list
persons_list = [
    '1_20160518.mat', '2_20150915.mat', '3_20150919.mat', '4_20151111.mat',
    '5_20160406.mat', '6_20150507.mat', '7_20150715.mat', '8_20151103.mat',
    '9_20151028.mat', '10_20151014.mat', '11_20150916.mat', '12_20150725.mat',
    '13_20151115.mat', '14_20151205.mat', '15_20150508.mat'
]

# Initialize the container for data
X = []

# Initialize the variable to track the maximum W (number of time windows)
max_W = 64

eeg_fields = ['de_LDS', 'de_movingAve', 'psd_LDS', 'psd_movingAve']

# Debug: Start processing
print("Starting to process .mat files...\n")

# Second pass to load data and pad the time windows to max_W
for file in persons_list:
    mat_file_path = os.path.join(data_dir, file)
    print(f"Checking file: {mat_file_path} --> Exists? {os.path.exists(mat_file_path)}")
    print(f"Processing file: {file}...")  # Debug: Current file

    if os.path.exists(mat_file_path):  # Check if the file exists
        print(f"  File {file} found, loading...\n")
        mat_data = scipy.io.loadmat(mat_file_path)
        
        # Initialize a container for the current person's data
        person_data = []

        # Iterate over each EEG field and pad the time windows if necessary
        for i in range(24):
            print(f"    Processing trial {i+1}...")  # Debug: Current trial
            trial_data = []
            
            for field in eeg_fields:
                key = f"{field}{i+1}"
               # print(f"\nAvailable keys in {file}: {list(mat_data.keys())}\n")
                if key in mat_data:  # Check if key exists in .mat file
                    data = mat_data[key]
                    W = data.shape[1]  # Number of time windows in this trial
                    
                    print(f"      Found key: {key}, Shape: {data.shape}")  # Debug: Data shape
                    
                    # If W is smaller than max_W, pad the time windows
                    if W < max_W:
                        padding_width = max_W - W
                        data_padded = np.pad(data, [(0, 0), (0, padding_width), (0, 0)], mode='constant')
                        print(f"      Padded {key} from W={W} to max_W={max_W}")  # Debug: Padding info
                    else:
                        data_padded = data  # No padding needed if W is already max_W
                    
                    trial_data.append(data_padded)
                else:
                    print(f"      Key missing: {key}, skipping...")  # Debug: Missing key
            
            X.append(trial_data)

    else:
        print(f"  Warning: File {file} not found, skipping...\n")  # Debug: File not found

# Debug: Processing finished
print("\nProcessing complete. Checking final dataset shape...\n")
print(f"Final shape of X: {np.shape(X)}")  # Debug: Output final shape


Checking directory: C:\Users\ASUS\Desktop\BTP\eeg_feature_smooth\1
Does directory exist? True
Starting to process .mat files...

Checking file: C:\Users\ASUS\Desktop\BTP\eeg_feature_smooth\1\1_20160518.mat --> Exists? True
Processing file: 1_20160518.mat...
  File 1_20160518.mat found, loading...

    Processing trial 1...
      Found key: de_LDS1, Shape: (62, 42, 5)
      Padded de_LDS1 from W=42 to max_W=64
      Found key: de_movingAve1, Shape: (62, 42, 5)
      Padded de_movingAve1 from W=42 to max_W=64
      Found key: psd_LDS1, Shape: (62, 42, 5)
      Padded psd_LDS1 from W=42 to max_W=64
      Found key: psd_movingAve1, Shape: (62, 42, 5)
      Padded psd_movingAve1 from W=42 to max_W=64
    Processing trial 2...
      Found key: de_LDS2, Shape: (62, 23, 5)
      Padded de_LDS2 from W=23 to max_W=64
      Found key: de_movingAve2, Shape: (62, 23, 5)
      Padded de_movingAve2 from W=23 to max_W=64
      Found key: psd_LDS2, Shape: (62, 23, 5)
      Padded psd_LDS2 from W=23 to 

In [26]:
(np.array(X)).shape

(360, 4, 62, 64, 5)

In [27]:
y_val = np.resize(np.array(y),360)
print(y_val)

[1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1
 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2
 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1 1 2
 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0
 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2
 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0
 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3
 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1
 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0
 3 0 3 1 2 3 0 2 0 0 1 0 1 2 1 1 1 2 3 2 2 3 3 0 3 0 3]


In [28]:
import os

session = 2
path = r"C:\Users\ASUS\Desktop\BTP\eeg_raw_data"  # Use raw string or double backslashes

# Construct the full path
session_path = os.path.join(path, str(session))

# Check if the directory exists
if os.path.exists(session_path):
    persons_list = os.listdir(session_path)
    print("Files:", persons_list)
else:
    print("Error: Directory does not exist:", session_path)

y_Val = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]

Files: ['10_20151021.mat', '11_20150921.mat', '12_20150804.mat', '13_20151125.mat', '14_20151208.mat', '15_20150514.mat', '1_20161125.mat', '2_20150920.mat', '3_20151018.mat', '4_20151118.mat', '5_20160413.mat', '6_20150511.mat', '7_20150717.mat', '8_20151110.mat', '9_20151119.mat']


In [29]:
import os
import scipy.io
import numpy as np

# Define the directory where the .mat files are stored
data_dir = r"C:\Users\ASUS\Desktop\BTP\eeg_feature_smooth\2"  # Corrected path for Windows

print(f"Checking directory: {os.path.abspath(data_dir)}")
print(f"Does directory exist? {os.path.exists(data_dir)}")

# Sample persons_list
persons_list = ['1_20161125.mat', '2_20150920.mat', '3_20151018.mat', '4_20151118.mat', '5_20160413.mat', '6_20150511.mat', 
 '7_20150717.mat', '8_20151110.mat', '9_20151119.mat', '10_20151021.mat', '11_20150921.mat', '12_20150804.mat', 
 '13_20151125.mat', '14_20151208.mat', '15_20150514.mat']

# Initialize the container for data
x_Val = []

# Initialize the variable to track the maximum W (number of time windows)
max_W = 64

eeg_fields = ['de_LDS', 'de_movingAve', 'psd_LDS', 'psd_movingAve']

# Debug: Start processing
print("Starting to process .mat files...\n")

# Second pass to load data and pad the time windows to max_W
for file in persons_list:
    mat_file_path = os.path.join(data_dir, file)
    print(f"Checking file: {mat_file_path} --> Exists? {os.path.exists(mat_file_path)}")
    print(f"Processing file: {file}...")  # Debug: Current file

    if os.path.exists(mat_file_path):  # Check if the file exists
        print(f"  File {file} found, loading...\n")
        mat_data = scipy.io.loadmat(mat_file_path)
        
        # Initialize a container for the current person's data
        person_data = []

        # Iterate over each EEG field and pad the time windows if necessary
        for i in range(24):
            print(f"    Processing trial {i+1}...")  # Debug: Current trial
            trial_data = []
            
            for field in eeg_fields:
                key = f"{field}{i+1}"
               # print(f"\nAvailable keys in {file}: {list(mat_data.keys())}\n")
                if key in mat_data:  # Check if key exists in .mat file
                    data = mat_data[key]
                    W = data.shape[1]  # Number of time windows in this trial
                    
                    print(f"      Found key: {key}, Shape: {data.shape}")  # Debug: Data shape
                    
                    # If W is smaller than max_W, pad the time windows
                    if W < max_W:
                        padding_width = max_W - W
                        data_padded = np.pad(data, [(0, 0), (0, padding_width), (0, 0)], mode='constant')
                        print(f"      Padded {key} from W={W} to max_W={max_W}")  # Debug: Padding info
                    else:
                        data_padded = data  # No padding needed if W is already max_W
                    
                    trial_data.append(data_padded)
                else:
                    print(f"      Key missing: {key}, skipping...")  # Debug: Missing key
            
            x_Val.append(trial_data)

    else:
        print(f"  Warning: File {file} not found, skipping...\n")  # Debug: File not found

# Debug: Processing finished
print("\nProcessing complete. Checking final dataset shape...\n")
print(f"Final shape of X: {np.shape(x_Val)}")  # Debug: Output final shape


Checking directory: C:\Users\ASUS\Desktop\BTP\eeg_feature_smooth\2
Does directory exist? True
Starting to process .mat files...

Checking file: C:\Users\ASUS\Desktop\BTP\eeg_feature_smooth\2\1_20161125.mat --> Exists? True
Processing file: 1_20161125.mat...
  File 1_20161125.mat found, loading...

    Processing trial 1...
      Found key: de_LDS1, Shape: (62, 55, 5)
      Padded de_LDS1 from W=55 to max_W=64
      Found key: de_movingAve1, Shape: (62, 55, 5)
      Padded de_movingAve1 from W=55 to max_W=64
      Found key: psd_LDS1, Shape: (62, 55, 5)
      Padded psd_LDS1 from W=55 to max_W=64
      Found key: psd_movingAve1, Shape: (62, 55, 5)
      Padded psd_movingAve1 from W=55 to max_W=64
    Processing trial 2...
      Found key: de_LDS2, Shape: (62, 25, 5)
      Padded de_LDS2 from W=25 to max_W=64
      Found key: de_movingAve2, Shape: (62, 25, 5)
      Padded de_movingAve2 from W=25 to max_W=64
      Found key: psd_LDS2, Shape: (62, 25, 5)
      Padded psd_LDS2 from W=25 to 

In [31]:

x_train = np.array(X)
y_train = tf.one_hot(np.resize(np.array(y),360),4)
x_val = np.array(x_Val)
y_val = tf.one_hot(np.resize(np.array(y_Val),360),4)




if not os.path.exists('./logs/' + file_name):
    os.makedirs('./logs/' + file_name)



# Convert x_val and y_val to numpy arrays if they are not already
x_val = np.array(x_val)
y_val = np.array(y_val)

csv_logger = CSVLogger('./logs/' + file_name + '/train_logger.csv', separator=',', append=True)

# Define the checkpoint path and filename
checkpoint_path = PREFIX_FOR_LOG+'./logs/'+ file_name +'/' + file_name + '.h5'

# Define the ModelCheckpoint callback
best_checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      monitor='val_subset_accuracy',
                                      save_best_only=True,
                                      save_weights_only=True,
                                      mode='max',
                                      verbose=1)

final_checkpoint_path = PREFIX_FOR_LOG+'./logs/'+ file_name +'/final_' + file_name + '.h5'
final_checkpoint_callback = ModelCheckpoint(final_checkpoint_path,
                                            save_weights_only=True,
                                            verbose=1)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate),
    metrics=[
        keras.metrics.Recall(), 
        keras.metrics.Precision(),
        keras.metrics.BinaryAccuracy(), 
        keras.metrics.FalseNegatives(),
        keras.metrics.AUC(curve='ROC'),
        keras.metrics.AUC(curve='PR'),
        subset_accuracy],
    loss= keras.losses.BinaryCrossentropy(from_logits=False)
)

model.summary(line_length=100)

print(model.predict(x_val))

model.evaluate(x_val, y_val)

history = model.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val),
    callbacks=[csv_logger, best_checkpoint_callback, final_checkpoint_callback], shuffle=True,
)

plot_training_history(history)

NameError: name 'file_name' is not defined